# 使い方
これはmatファイルをptファイルに変換するノートブックです。<br>
基本的には上から順に一つずつ実行していくことで、当該ファイルの変換がされます。<br>
自分が設定しなければならないのは
1. --- 定数の設定 ---
2. --- keyの設定 --- 

--- 定数の設定 ---は、主にファイルのPATH指定になります。<br>
--- keyの設定 ---は、matファイルに保存されたキーを--- keyの取得 ---で確認し、設定します。

In [ ]:
import os
import numpy as np
import h5py
import torch
import scipy.io as sio

In [ ]:
# --- 定数の設定 ---
# .matのデータフォルダ場所
MAT_PATH = "/mnt/c/Users/onion/Documents/data"

# .ptのデータフォルダ保存場所
PT_PATH = "/mnt/c/Users/onion/Documents/processed_data"

# processed_dataのフォルダ構成
PT_FOLDERS = ["train", "val", "test", "target"]

# pathのディレクトリ部分が存在しない場合は作成、存在してもエラーにしない
for PT_FOLDER in PT_FOLDERS:
    os.makedirs(os.path.join(PT_PATH, PT_FOLDER), exist_ok=True)
    print(f"{os.path.join(PT_PATH, PT_FOLDER)} が作成されました。")

# targetのデータ名
TARGET_NAMES = ["m_train", "m_val", "m_test", "kappa_train", "kappa_val", "kappa_test"]

# .matのstructureデータフォルダ場所
STRUCTURE_MAT_PATHS = [os.path.join(MAT_PATH, "structures_train"),
                      os.path.join(MAT_PATH, "structures_val"),
                      os.path.join(MAT_PATH, "structures_test")]

# .matのstructureファイル数
STRUCTURE_MAT_NUMS = [len(os.listdir(STRUCTURE_MAT_PATHS[0])),
                      len(os.listdir(STRUCTURE_MAT_PATHS[1])),
                      len(os.listdir(STRUCTURE_MAT_PATHS[2]))]
print(STRUCTURE_MAT_NUMS)


/mnt/c/Users/onion/Documents/processed_data/train が作成されました。
/mnt/c/Users/onion/Documents/processed_data/val が作成されました。
/mnt/c/Users/onion/Documents/processed_data/test が作成されました。
/mnt/c/Users/onion/Documents/processed_data/target が作成されました。
[63000, 13500, 13500]


In [ ]:
# --- keyの取得 ---
# targetのkeysを取得
for TARGET_NAME in TARGET_NAMES:
    target_mat_path = os.path.join(MAT_PATH, f"{TARGET_NAME}.mat")

    with h5py.File(target_mat_path, "r") as f:
        keys = list(f.keys())
        print(f"{TARGET_NAME}.mat のkeyは {keys} が設定されているよ")

# 初めの3つのstructureのkeysを取得
for i in range(3):
    train_structure_mat_path = os.path.join(STRUCTURE_MAT_PATHS[0], f"structure_{i+1}.mat")
    data = sio.loadmat(train_structure_mat_path)
    keys = list(data.keys())
    print(f"structure_{i}.mat のkeyは {keys} が設定されているよ")

m_train.mat のkeyは ['y'] が設定されているよ
m_val.mat のkeyは ['y'] が設定されているよ
m_test.mat のkeyは ['y'] が設定されているよ
kappa_train.mat のkeyは ['y'] が設定されているよ
kappa_val.mat のkeyは ['y'] が設定されているよ
kappa_test.mat のkeyは ['y'] が設定されているよ
structure_0.mat のkeyは ['__header__', '__version__', '__globals__', 'BW'] が設定されているよ
structure_1.mat のkeyは ['__header__', '__version__', '__globals__', 'BW'] が設定されているよ
structure_2.mat のkeyは ['__header__', '__version__', '__globals__', 'BW'] が設定されているよ


In [ ]:
# --- keyの設定 ---
# 上記のコードから取得したキーからほしいデータだけを指定する
# ただし、キーがすべて同じである前提である
TARGET_KEY = 'y'
STRUCTURE_KEY = 'BW'

In [ ]:
# --- targetのmat2pt ---
for TARGET_NAME in TARGET_NAMES:
    target_mat_path = os.path.join(MAT_PATH, f"{TARGET_NAME}.mat")
    target_pt_path = os.path.join(PT_PATH, PT_FOLDERS[3], f"{TARGET_NAME}.pt")

    # MATLABファイルを読み込み、指定された変数をPyTorchのテンソルに変換して保存
    with h5py.File(target_mat_path, "r") as f:
        data = f[TARGET_KEY]
        data = np.array(data).T
        data = torch.tensor(data)
        torch.save(data, target_pt_path)

        if os.path.exists(target_pt_path):
            print(f"{TARGET_NAME}.pt が作成されました。")

m_train.pt が作成されました。
m_val.pt が作成されました。
m_test.pt が作成されました。
kappa_train.pt が作成されました。
kappa_val.pt が作成されました。
kappa_test.pt が作成されました。


In [ ]:
# --- structureのmat2pt ---
for i in range(3): # [train, val, test]の3つ
    for j in range(STRUCTURE_MAT_NUMS[i]):
        structure_mat_path = os.path.join(STRUCTURE_MAT_PATHS[i], f"structure_{j+1}.mat")
        structure_pt_path = os.path.join(PT_PATH, PT_FOLDERS[i], f"structure_{j+1}.pt")

        data = sio.loadmat(structure_mat_path)
        data = data[STRUCTURE_KEY]
        data = np.array(data).T
        data = torch.tensor(data)
        torch.save(data, structure_pt_path)